In [16]:
import pandas as pd
import requests
import html5lib
from bs4 import BeautifulSoup
import urllib.request
import string
import re
import random
import time
from collections import Counter

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

## NIH - List

No stop words necessary

In [5]:
url = u'https://rarediseases.info.nih.gov/diseases/fda-orphan-drugs/'

In [6]:
string.ascii_uppercase

'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

In [7]:
all_orphans = []
for ending in string.ascii_uppercase:
    print("processing %s" % ending)
    cur_url = url + ending
    r = requests.get(cur_url)
    soup = BeautifulSoup(r.text, 'html5lib')
    all_a = soup.find_all('a', attrs={"class": "DiseaseClick"})
    print("%s found ..." % len(all_a))
    
    for cur_a in all_a:
        all_orphans.append(cur_a.text.lower().strip())
        
    time.sleep(random.uniform(1, 2))

processing A
39 found ...
processing B
11 found ...
processing C
42 found ...
processing D
11 found ...
processing E
9 found ...
processing F
19 found ...
processing G
12 found ...
processing H
39 found ...
processing I
5 found ...
processing J
2 found ...
processing K
2 found ...
processing L
16 found ...
processing M
37 found ...
processing N
17 found ...
processing O
15 found ...
processing P
42 found ...
processing Q
0 found ...
processing R
5 found ...
processing S
20 found ...
processing T
18 found ...
processing U
2 found ...
processing V
4 found ...
processing W
3 found ...
processing X
1 found ...
processing Y
0 found ...
processing Z
1 found ...


In [8]:
len(all_orphans)

372

## FDA - approved list

here we need to figure out stopwords

In [2]:
fp = r'xml/raw_fda.html'

In [3]:
soup = BeautifulSoup(open(fp, encoding='UTF-8'), 'html5lib')
print(soup.prettify()[:1000])

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3c.org/TR/1999/REC-html401-19991224/loose.dtd">
<html xmlns="http://www.w3.org/1999/xhtml" xmlns:addthis="http://www.addthis.com/help/api-spec">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="IE=7" http-equiv="X-UA-Compatible"/>
  <script src="/scripts/includes/js/wcm.toggle.js" type="text/javascript">
  </script>
  <script src="/scripts/includes/js/ssajax_2012.js" type="text/javascript">
  </script>
  <script language="JavaScript" src="fsrscripts\listResult.js" type="text/javascript">
  </script>
  <title>
   Search Orphan Drug Designations and Approvals
  </title>
  <!-- /**** Begin CSS References ****/-->
  <link href="/scripts/includes/css/css_fda_gov_stylesheet-2013.css" media="screen" rel="stylesheet" type="text/css"/>
  <link href="/scripts/includes/css/css_fda_gov_print_stylesheet-2013.css" media="print" rel="stylesheet" type="text/css"/>
  <!--[if lt IE 

In [4]:
tables = soup.find_all('table')
datatable = tables[2]
df = pd.read_html(str(datatable))[0]
df.columns = ['id', 'gen_name', 'orph', 'desig_dt', 'desig_status']
df.head()

,id,gen_name,orph,desig_dt,desig_status
0,1,"((1r, 4r)-N1-(2-benzyl-7-(2-methyl-2H-tetrazol...",Prevention of Graft-versus-Host-Disease,12/13/2018,Designated
1,2,(+)-(2S)-2-(4-chloro-2-methoxyphenyl)-2-{[3-me...,Treatment of dengue virus infection,12/26/2017,Designated
2,3,(+)-1-(3-(aminomethyl) phenyl)-N-(5-((3-cyanop...,Treatment of C1-inhibitor-dependent angioedema...,11/01/2017,Designated
3,4,(+)-a-dihydrotetrabenazine,Treatment pediatric patients with Tourette's s...,03/30/2017,Designated
4,5,(+)-alpha-dihydrotetrabenazine,Treatment of Huntington's disease.,12/05/2018,Designated


In [14]:
df.shape

(5165, 5)

In [10]:
df['desig_status'].unique()

array(['Designated', 'Designated/Withdrawn', 'Designated/Approved',
       'Designated/Approved/Withdrawn'], dtype=object)

In [27]:
df_desig = df[df['desig_status'].isin(['Designated', 'Designated/Approved'])].copy()
df_desig.shape

(4436, 6)

In [28]:
# let's count our tokens to figure out the stop words
c = Counter()

for cur_str in df_desig['orph']:
    tokens = cur_str.split(' ')
    for cur_token in tokens:
        processed = cur_token.lower().translate(str.maketrans('', '', string.punctuation))
        c[processed] += 1

In [29]:
c.most_common(100)

[('of', 4645),
 ('treatment', 4013),
 ('in', 608),
 ('with', 494),
 ('patients', 485),
 ('and', 419),
 ('the', 404),
 ('disease', 403),
 ('cancer', 387),
 ('for', 359),
 ('syndrome', 321),
 ('to', 294),
 ('acute', 277),
 ('leukemia', 267),
 ('cell', 233),
 ('or', 218),
 ('prevention', 213),
 ('lymphoma', 207),
 ('chronic', 163),
 ('pediatric', 149),
 ('carcinoma', 139),
 ('pancreatic', 131),
 ('pulmonary', 130),
 ('fibrosis', 128),
 ('sclerosis', 123),
 ('a', 121),
 ('deficiency', 117),
 ('multiple', 116),
 ('use', 113),
 ('type', 113),
 ('myeloid', 112),
 ('stage', 109),
 ('melanoma', 98),
 ('myeloma', 96),
 ('malignant', 94),
 ('ovarian', 90),
 ('cystic', 87),
 ('associated', 85),
 ('muscular', 84),
 ('lung', 83),
 ('primary', 80),
 ('dystrophy', 79),
 ('therapy', 79),
 ('iv', 74),
 ('tumors', 73),
 ('graft', 73),
 ('as', 73),
 ('idiopathic', 72),
 ('glioma', 71),
 ('lateral', 70),
 ('lymphocytic', 69),
 ('infection', 68),
 ('amyotrophic', 68),
 ('glioblastoma', 64),
 ('due', 64),
 (

In [30]:
stop_words = [
    'of',
    'treatment',
    'in',
    'with',
    'patients',
    'and',
    'the',
    'for',
    'to',
    'disease',
    'or',
    'prevention',
    'use',
    'a',
    'as',
    'who',
    'by',
    'management',
    'through',
    'including',
    'iib',
    'diagnostic',
    'be',
    'used'
]

In [31]:
processed_col = []
for cur_str in df_desig['orph']:
    tokens = cur_str.split(' ')
    processed_tokens = []
    for cur_token in tokens:
        cur_processed = cur_token.lower().translate(str.maketrans('', '', string.punctuation))
        if cur_processed not in stop_words:
            processed_tokens.append(cur_processed)
    full_str = ' '.join(processed_tokens)
    processed_col.append(full_str)

In [32]:
df_desig['processed'] = processed_col

In [33]:
df_counts = df_desig.groupby('processed')[['id']].count().sort_values('id', ascending=False).reset_index()
df_counts = df_counts.rename(columns={'id': 'count'})
df_counts.head()

,processed,count
0,pancreatic cancer,109
1,multiple myeloma,88
2,acute myeloid leukemia,84
3,ovarian cancer,68
4,cystic fibrosis,56


## combining the 2 lists

In [34]:
for cur_val in df_counts['processed'].values:
    clean_val = cur_val.strip()
    if clean_val not in all_orphans:
        all_orphans.append(clean_val)

In [36]:
len(all_orphans)

2488

In [37]:
import pickle

In [40]:
pickle.dump(all_orphans, open('raw_data/orphan_list.pkl', 'wb'))